In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, make_scorer

# Filepath 
file_path = r'INSERT HERE'
df = pd.read_excel(file_path)

#Define Features & Target
X = df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
        'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
        'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27']]
y = df['status']

# Splitting 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99, stratify=y)

# XGB
xg_clf = xgb.XGBClassifier(objective='binary:logistic', colsample_bytree=0.3, learning_rate=0.1,
                           max_depth=5, alpha=10, n_estimators=10)

# AUC
scorer = make_scorer(roc_auc_score)

# Defining Parameter
param_grid = {
    'learning_rate': [0.1, 0.01, 0.05],
    'max_depth': [1, 3, 5,],
    'n_estimators': [50, 100, 150, 200]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=xg_clf, param_grid=param_grid, cv=5, scoring=scorer)

# Deploy
grid_search.fit(X_train, y_train)

# Print Result
print("Best Parameters:", grid_search.best_params_)
print("Best AUC-ROC Score:", grid_search.best_score_)

# Deploy XGB
best_xg_clf = grid_search.best_estimator_

# Predicting
y_pred = best_xg_clf.predict(X_test)
y_pred_proba = best_xg_clf.predict_proba(X_test)[:, 1]

# Results
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("Final AUC-ROC Score:", auc_roc)
accuracy = accuracy_score(y_test, y_pred)
print("Final Accuracy:", accuracy)
